In [ ]:
'''     terminal run - for 1st run
pip install numpy
pip install pandas
pip install openpyxl
'''

#  !!!  use  kernel not less than Python 3.10.7  (Jupiter's issue - earlier versions dosn't support it )

#to-do list
#  !   add a sort-out if some file is NOT a JSON-messages file  ( file-name is not like a "yyyy-mm-dd.json" )
#  !   provide defining an Excel's columns "Letter" by "column's-title"  ( to prewent hardcoding of Excel'scolumns "Letters" to format )
#  !   correct date-entrance ::  "json_mod_date"  comes with wrong date
#  !   add a fork (or 1 - or all channels to convert): if "exportname" exist and "exportname"<>""  =>>  code run over one single folder named "exportname"
#         otherwise code iterate over ALL of like-channel's-folder inside starting root(export) folder == "working_directory"
#  !   add a excel-column and code to extract hyperlink(s) from "text" (if h/l presen) 
#  !   add a excel-column and code to extract hyperlink(s) from other parts of message  (if h/l presen) 
#  !   add a keyword(s)'s evaluation and extraction/separation to the proper columns added  
#  !   add a substitutions for channel-IDs and user_IDs in the "text" and in the other parts of messages (if IDs is/are present)
#  !   adjust timestamps to the CST-time-zone and provide variable for easy-hard-code this thing
#  !   add check-and-fix "if the export directory exist" :: if "no" >> then create it


import os
import json
import numpy as np
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.styles import PatternFill


# how many rows demonstrate in executional cells 
rows_to_show = 1   # uses for debug with Jupiter's-cells-system

exportname = "3d-art-blender-landmarks"        #Insert the certain "desired" for convert Channel-Name here
# should be added to code ::
#  if "exportname" exist and "exportname"<>""  =>>  code run over one single folder named "exportname"
#    otherwise code iterate over ALL of like-channel's-folder inside starting root(export) folder == "working_directory"

#        Insert path where the LOCAL copy of the GoogleDrive folder is.
working_directory = 'E:\_RET_slack_export\RebeccaEverlene Slack export Apr 30 2021 - Oct 3 2024-2' 
exporting_directory = f"{working_directory}\{'_converted'}" 
#"E:\_RET_slack_export" #os.getcwd()
#
slackexport_folder_path = f"{working_directory}\{exportname}"  #  for future use with "exportname" fork will be coded
#slackexport_folder_path = "/home/agds/Documents/RebeccaEverleneTrust/RebeccaEverlene_Slack_export"  #
#print("slackexport_folder_path= "+slackexport_folder_path)


#
#
### 1. Retrieving all information from Slack's channels and users:

# The code presented in this section was initially developed by Rutvikk Kharod (https://github.com/Rutvikk-Khar/Data-Handling-of-Slack-JSON-Exports). 
# There are some modifications with respect to the original version, made by @Angelica Goncalves Dos Santos & @Iakov 
#   mainly to include some extra information in the dataframes and the further csv/xlsx files. Export shifted to xlsx's and xlsx's a bit beautified. 


## 1.1. Slack channels:
# Reads the json file "channels.json",
#   transfers the general information of each Slack channel to a dataframe 
# and writes it to the "all_channels.csv" file.

channels_path = f"{working_directory}\channels.json"
#
#print("channels_path = "+channels_path)
with open(channels_path, encoding='utf-8') as f:
    channels_json = json.load(f)

channel_list = pd.DataFrame(columns=["ch_id", "name", "created", "creator", "is_archived",
                                     "is_general", "members", "topic", "purpose",'dayslist'])

for channel in range(len(channels_json)):
    channel_list.at[channel, "ch_id"] = channels_json[channel]['id']
    channel_list.at[channel, "name"] = channels_json[channel]['name']
    channel_list.at[channel, "created"] = channels_json[channel]['created']
    channel_list.at[channel, "creator"] = channels_json[channel]['creator']
    channel_list.at[channel, "is_archived"] = channels_json[channel]['is_archived']
    channel_list.at[channel, "is_general"] = channels_json[channel]['is_general']
    memberlist = ", ".join(channels_json[channel]['members'])
    channel_list.at[channel, "members"] = memberlist
    channel_list.at[channel, "topic"] = channels_json[channel]['topic']['value']
    # "purpose"  should be done as string, not a list
    channel_list.at[channel, "purpose"] = ''.join(channels_json[channel]['purpose']['value'])
    
    channel_folder_path = f"{working_directory}/{channel_list.at[channel, 'name']}"
    #print("channel_folder_path = "+ channel_folder_path)
    if os.path.exists((channel_folder_path)):
        #print("os.listdir(channel_folder_path) = "+' '.join(os.listdir(channel_folder_path)))
        channel_list.at[channel, "dayslist"] = os.listdir(channel_folder_path)

        #   !   add a sort-out if some file is NOT a JSON-messages file  ( file-name is not like a "yyyy-mm-dd.json" )
        
        channels_json[channel]['dayslist'] = os.listdir(channel_folder_path)
    else:
        channel_list.at[channel, "dayslist"] ="absent"
        channels_json[channel]['dayslist'] = "absent"
    #channels_json[channel]['dayslist'] = os.listdir(channel_folder_path)
# and writes it to the "all_channels.csv" file.
slack_export_channel_filename = "_all_channels"
slack_export_channel__folder_path = f"{working_directory}/{slack_export_channel_filename}{'.csv'}"
#channel_list.to_csv(slack_export_channel__folder_path, index=False)
# 
slack_export_channel__folder_path_xlsx = f"{exporting_directory}/{slack_export_channel_filename}{'.xlsx'}"
channel_list.to_excel(slack_export_channel__folder_path_xlsx, index=False)
#
#print("slack_export_channel__folder_path => "+slack_export_channel__folder_path)

#channel_list[:rows_to_show]  


## 1.2. Slack users:
# Reads the json file "users.json", 
#    transfers the general information of each Slack user to a dataframe 
#    and writes it to the "all_users.csv" file.

users_path = f"{working_directory}/users.json"
with open(users_path, encoding='utf-8') as f:
    users_json = json.load(f)

user_list_df = pd.DataFrame(columns=["user_id", "team_id", "name", "deleted", "real_name",
                                      "tz", "tz_label", "tz_offset", "title", "display_name", 
                                      "is_bot"])

for user in range(len(users_json)):
    user_list_df.at[user, "user_id"] = users_json[user]['id']
    user_list_df.at[user, "team_id"] = users_json[user]['team_id']
    user_list_df.at[user, "name"] = users_json[user]['name']
    user_list_df.at[user, "deleted"] = users_json[user]['deleted']
    user_list_df.at[user, "real_name"] = users_json[user].get('real_name', 'n/a') #None) # 20241110-2
    user_list_df.at[user, "title"] = users_json[user]['profile']['title']
    user_list_df.at[user, "display_name"] = users_json[user]['profile']['display_name']
    user_list_df.at[user, "is_bot"] = users_json[user]['is_bot']
    user_list_df.at[user, "tz"] = users_json[user].get('tz',  'n/a') #None)
    user_list_df.at[user, "tz_label"] = users_json[user].get('tz_label',  'n/a') #None)
    user_list_df.at[user, "tz_offset"] = users_json[user].get('tz_offset',  'n/a') #None)

slack_export_user_filename = "_all_users"
slack_export_user__folder_path = f"{working_directory}/{slack_export_user_filename}{'.csv'}"
#user_list_df.to_csv(slack_export_user__folder_path, index=False)
#

slack_export_user__folder_path_xlsx = f"{exporting_directory}/{slack_export_user_filename}{'.xlsx'}" #_IP
user_list_df.to_excel(slack_export_user__folder_path_xlsx, index=False) #_IP
#
#print("slack_export_user__folder_path = "+slack_export_user__folder_path)

#user_list_df[:rows_to_show]


## 1.3. Slack messages:
# Collects all the messages in "RebeccaEverlene_Slack_export", 
#   stores them on the data frame "all_messages_df" 
#   and writes it into the file "all_channels_{Begin_Date}_{End_Date}.csv":
def slack_json_to_dataframe(slack_json):
    messages_df = pd.DataFrame(columns=["msg_id", "ts", "user", "type", "text", 
                                        "reply_count", "reply_users_count", 
                                        "ts_latest_reply", "ts_thread", "parent_user_id"])
    
    for message in range(len(slack_json)):
        if 'files' in slack_json[message] and slack_json[message]['files']:
            messages_df.at[message, "msg_id"] = slack_json[message]['files'][0]['id']
        elif 'client_msg_id' in slack_json[message]:
            messages_df.at[message, "msg_id"] = slack_json[message]['client_msg_id']
        else:
            messages_df.at[message, "msg_id"] = 'n/a'

        if 'ts' in slack_json[message]:
            messages_df.at[message, "ts"] = slack_json[message]['ts']
        else:
            messages_df.at[message, "ts"] = 'n/a' #None # 20241110-2

        messages_df.at[message, "user"] = slack_json[message].get('user', 'n/a') #None) # 20241110-2

        if 'type' in slack_json[message]:
            messages_df.at[message, "type"] = slack_json[message]['type']
        else:
            messages_df.at[message, "type"] = 'n/a' #None # 20241110-2

        if 'text' in slack_json[message]:
            messages_df.at[message, "text"] = slack_json[message]['text']
        else:
            messages_df.at[message, "text"] = 'n/a' #None # 20241110-2

        if 'reply_count' in slack_json[message]:
            messages_df.at[message, "reply_count"] = slack_json[message]['reply_count']
            messages_df.at[message, "reply_users_count"] = slack_json[message]['reply_users_count']
            messages_df.at[message, "ts_latest_reply"] = slack_json[message]['latest_reply']
        else:
            messages_df.at[message, "reply_count"] = 'n/a'  # 20241110-2
            messages_df.at[message, "reply_users_count"] = 'n/a'  # 20241110-2
            messages_df.at[message, "ts_latest_reply"] = 'n/a'  # 20241110-2

        if 'parent_user_id' in slack_json[message]:
            messages_df.at[message, "ts_thread"] = slack_json[message]['thread_ts']
            messages_df.at[message, "parent_user_id"] = slack_json[message]['parent_user_id']
        else:
            messages_df.at[message, "ts_thread"] = 'n/a'  # 20241110-2
            messages_df.at[message, "parent_user_id"] = 'n/a'  # 20241110-2

    return messages_df


# iterate over channel's folders 
for channel in range(len(channels_json)):
    #print("channel ->>  "+str(channel))
    
    # evaluate starts only if current channel's folder exists
    curr_channel_name = channels_json[channel]['name']    # IP 20241110
    print("channel ->>  "+str(channel)+"   curr_channel_name ->>  "+curr_channel_name)
    
    parentfolder_path = f"{working_directory}/{curr_channel_name}" 
    #print("-> parentfolder_path = " + parentfolder_path)

    if os.path.exists(parentfolder_path):   #   (f"{working_directory}/{curr_channel_name}"):
        print("channel's_folder ->>  "+f"{working_directory}/{curr_channel_name}")
        #print("channel's_folder ->>  "+f"{working_directory}/{curr_channel_name}")
        channel_messages_df = pd.DataFrame(columns=["msg_id", "ts", "user", "type", "text",
                                                  "reply_count", "reply_users_count",
                                                  "ts_latest_reply", "ts_thread", "parent_user_id"])
                                                 # ,"channel_folder", "json_name", "json_mod_date"])          #_IP
        
        # iterate over JSONs inside the current channel's folder 
        for file_day in range(len(channels_json[channel]['dayslist'])):
            filejson_path = f"{parentfolder_path}/{channels_json[channel]['dayslist'][file_day]}"

            with open(filejson_path, encoding='utf-8') as f:
                import_file_json = json.load(f)
            import_file_df = slack_json_to_dataframe(import_file_json)
            import_file_df['json_name'] = channels_json[channel]['dayslist'][file_day]                 #[:-5]
            import_file_df['json_mod_date'] = datetime.fromtimestamp(os.path.getmtime(filejson_path))
            #print(channels_json[channel]['dayslist'][file_day], datetime.fromtimestamp(os.path.getmtime(filejson_path)))
         
            channel_messages_df = pd.concat([channel_messages_df, import_file_df], axis=0, ignore_index=True)    #IP //  ip>>20241111    , axis=0
        # end of for-"file_day"  iteration

        # for current channel iterated :
        channel_messages_df['channel_folder'] = curr_channel_name  #IP
        #channel_messages_df = pd.concat([channel_messages_df, channel_files_df], ignore_index=True) #IP
        #
        #     Use the "user_list_df" dataframe to fill-in the user's information for each message:
        for index in channel_messages_df.index.values:    #_IP
            #channel_messages_df.at[index, 'user_id'] = user_list_df[user_list_df['user_id']==channel_messages_df.loc[index]['user']]['user_id'].values
            #channel_messages_df.at[index, 'team_id'] = user_list_df[user_list_df['user_id']==channel_messages_df.loc[index]['user']]['team_id'].values
            channel_messages_df.at[index, 'user_name'] = user_list_df[user_list_df['user_id']==channel_messages_df.loc[index]['user']]['name'].values
            channel_messages_df.at[index, 'display_name'] = user_list_df[user_list_df['user_id']==channel_messages_df.loc[index]['user']]['display_name'].values
        #  end of for-"index"
        #
        #  Change format of the time in seconds to a date:    _IP
        #channel_messages_df['ts'] = pd.to_datetime(channel_messages_df['ts'], unit='s')
        #channel_messages_df.rename(columns={"ts": "msg_date"}, inplace=True)
        #channel_messages_df.sort_values(by='msg_date', inplace=True)

        #
        # "channel_messages_filename" - filename compilation using start and finish date for messages-in-the-channel
        channel_messages_df['ts'] = pd.to_numeric(channel_messages_df['ts'], errors='coerce')   #_IP
        channel_messages_mindate = pd.to_datetime(channel_messages_df['ts'], unit='s').min().date()   #_IP
        channel_messages_maxdate = pd.to_datetime(channel_messages_df['ts'], unit='s').max().date()   #_IP

        channel_messages_df['ts'] = pd.to_datetime(channel_messages_df['ts'], unit='s')   #_IP
        channel_messages_df.rename(columns={"ts": "msg_date"}, inplace=True)   

        channel_messages_filename = f"{channels_json[channel]['name']}_{channel_messages_mindate}_to_{channel_messages_maxdate}" #_IP 
        #channel_messages_filename_xlsx = f"{channels_json[channel]['name']}_{channel_messages_mindate}_to_{channel_messages_maxdate}.xlsx" #_IP 
        #
        #print("channel_messages_filename -> "+channel_messages_filename)    #_IP
        #
        # To write a  .csv / .xlsx  file with current channel:
        #channel_messages__folder_path = f"{working_directory}/{channel_messages_filename}{'.csv'}"   #IP
        channel_messages__folder_path_xlsx = f"{exporting_directory}/{channel_messages_filename}{'.xlsx'}"   #IP
        #channel_messages_df.to_csv(channel_messages__folder_path, index=False)     #IP
        channel_messages_df.to_excel(channel_messages__folder_path_xlsx, index=False)
        #
        #print("channel_messages__folder_path_xlsx -> "+channel_messages__folder_path_xlsx)    #IP
        #
        #   >>>     Excel file formatting/adjustments with  openpyxl
        wb = load_workbook(channel_messages__folder_path_xlsx)
        ws = wb.active
        # Set the column width
        column_widths = {
            'B': 19, 'C': 15, 'E': 25, 'K': 25, 'L': 19, 'M': 19, 'N': 13, 'O': 13     
        }
        # Apply the column widths
        for col, width in column_widths.items():
            ws.column_dimensions[col].width = width
        #
        # Freeze the first row (Row 1)
        ws.freeze_panes = 'A2'
        # Set font size and bold for the first row
        font = Font(size=9, bold=True)
        # Define the RGB color
        fill = PatternFill(start_color="e7c9fb", end_color="e7c9fb", fill_type="solid")
        # Apply the color to the first row (row 1)
        # Apply the font formatting to the first row (Header row)
        for cell in ws[1]:
            cell.font = font
            cell.fill = fill
        #
        # Rename the sheet
        ws_title = channels_json[channel]['name'] 
        ws_title = ws_title[:31]
        ws.title = ws_title 
        #
        # Save the changes to the Excel file
        wb.save(channel_messages__folder_path_xlsx)
    #
    # end if "os.path.exists"
#
# end of for "for-channel" iteration